# ReadMe
If any code is not running, try re-install
```
langchain==0.0.316
openai==0.28.1
```

Documentation link: https://python.langchain.com/docs/use_cases/apis/

In [1]:
from dotenv import load_dotenv
import os

load_dotenv('.env', override=True)

True

In [2]:
from langchain.chains.openai_functions.openapi import get_openapi_chain

chain = get_openapi_chain(
    "https://www.klarna.com/us/shopping/public/openai/v0/api-docs/"
)
chain("What are some options for a men's large blue button down shirt")

Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


{'query': "What are some options for a men's large blue button down shirt",
 'response': {'products': [{'name': 'Izod Saltwater Big and Tall Mens Classic Fit Short Sleeve Button-Down Shirt, 3x-large, Blue Blue',
    'url': 'https://www.klarna.com/us/shopping/pl/cl10001/3263742310/Clothing/Izod-Saltwater-Big-and-Tall-Mens-Classic-Fit-Short-Sleeve-Button-Down-Shirt-3x-large-Blue-Blue/?utm_source=openai&ref-site=openai_plugin',
    'price': '$33.74',
    'attributes': ['Target Group:Man', 'Color:Blue']},
   {'name': "Izod Men's Saltwater Dockside Printed Chambray Button-Down Shirt, Large, Blue",
    'url': 'https://www.klarna.com/us/shopping/pl/cl10001/3257202629/Clothing/Izod-Men-s-Saltwater-Dockside-Printed-Chambray-Button-Down-Shirt-Large-Blue/?utm_source=openai&ref-site=openai_plugin',
    'price': '$29.99',
    'attributes': ['Target Group:Man',
     'Color:Blue',
     'Size (Small-Large):S,XL,L,M,XXL']},
   {'name': 'Cubavera Mens Classic Fit Short Sleeve Button-Down Shirt, X-large,

In [3]:
from langchain.chains.api import open_meteo_docs
print(open_meteo_docs.OPEN_METEO_DOCS)

BASE URL: https://api.open-meteo.com/

API Documentation
The API endpoint /v1/forecast accepts a geographical coordinate, a list of weather variables and responds with a JSON hourly weather forecast for 7 days. Time always starts at 0:00 today and contains 168 hours. All URL parameters are listed below:

Parameter	Format	Required	Default	Description
latitude, longitude	Floating point	Yes		Geographical WGS84 coordinate of the location
hourly	String array	No		A list of weather variables which should be returned. Values can be comma separated, or multiple &hourly= parameter in the URL can be used.
daily	String array	No		A list of daily weather variable aggregations which should be returned. Values can be comma separated, or multiple &daily= parameter in the URL can be used. If daily weather variables are specified, parameter timezone is required.
current_weather	Bool	No	false	Include current weather conditions in the JSON output.
temperature_unit	String	No	celsius	If fahrenheit is set, al

In [6]:
from langchain.chains import APIChain
from langchain.chains.api import open_meteo_docs
from langchain.llms import OpenAI

llm = OpenAI(temperature=0, model="gpt-3.5-turbo-instruct")
chain = APIChain.from_llm_and_api_docs(
    llm,
    open_meteo_docs.OPEN_METEO_DOCS,
    verbose=True,
    limit_to_domains=["https://api.open-meteo.com/"],
)
chain.run(
    "What is the weather like right now in Munich, Germany in degrees Fahrenheit?"
)



> Entering new APIChain chain...
 https://api.open-meteo.com/v1/forecast?latitude=48.1351&longitude=11.5820&hourly=temperature_2m&current_weather=true&temperature_unit=fahrenheit&timezone=auto
{"latitude":48.14,"longitude":11.58,"generationtime_ms":0.0959634780883789,"utc_offset_seconds":7200,"timezone":"Europe/Berlin","timezone_abbreviation":"CEST","elevation":521.0,"current_weather_units":{"time":"iso8601","interval":"seconds","temperature":"°F","windspeed":"km/h","winddirection":"°","is_day":"","weathercode":"wmo code"},"current_weather":{"time":"2024-04-15T20:45","interval":900,"temperature":56.9,"windspeed":13.7,"winddirection":256,"is_day":0,"weathercode":2},"hourly_units":{"time":"iso8601","temperature_2m":"°F"},"hourly":{"time":["2024-04-15T00:00","2024-04-15T01:00","2024-04-15T02:00","2024-04-15T03:00","2024-04-15T04:00","2024-04-15T05:00","2024-04-15T06:00","2024-04-15T07:00","2024-04-15T08:00","2024-04-15T09:00","2024-04-15T10:00","2024-04-15T11:00","2024-04-15T12:00","202

' The current temperature in Munich, Germany is 56.9 degrees Fahrenheit. This information was obtained from the API url: https://api.open-meteo.com/v1/forecast?latitude=48.1351&longitude=11.5820&hourly=temperature_2m&current_weather=true&temperature_unit=fahrenheit&timezone=auto.'

**Web requests**

URL requests are such a common use-case that we have the `LLMRequestsChain`, which makes an HTTP GET request. 

In [10]:
from langchain.chains import LLMChain, LLMRequestsChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

In [11]:
template = """Between >>> and <<< are the raw search result text from google.
Extract the answer to the question '{query}' or say "not found" if the information is not contained.
Use the format
Extracted:<answer or "not found">
>>> {requests_result} <<<
Extracted:"""

PROMPT = PromptTemplate(
    input_variables=["query", "requests_result"],
    template=template,
)


In [14]:
chain = LLMRequestsChain(llm_chain=LLMChain(llm=OpenAI(temperature=0, model="gpt-3.5-turbo-instruct"), prompt=PROMPT))
question = "What are the Three (3) biggest countries, and their respective sizes?"
inputs = {
    "query": question,
    "url": "https://www.google.com/search?q=" + question.replace(" ", "+"),
}
chain(inputs)

{'query': 'What are the Three (3) biggest countries, and their respective sizes?',
 'url': 'https://www.google.com/search?q=What+are+the+Three+(3)+biggest+countries,+and+their+respective+sizes?',
 'output': '1. Russia - 17,098,242 km\n2. Canada - 9,984,670 km\n3. United States - 9,826,675 km'}